In [1]:
pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.1 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#imports
import netCDF4
import xarray as xr
import numpy as np
import pandas as pd


# FLDAS

In [4]:
file_path_nc ="/content/drive/MyDrive/Thesis/RAW/Combined_FLDAS.nc"
dataset = xr.open_dataset(file_path_nc)
dataset


<xarray.Dataset> Size: 141MB
Dimensions:                (time: 251, bnds: 2, X: 160, Y: 110)
Coordinates:
  * time                   (time) datetime64[ns] 2kB 2003-02-01 ... 2023-12-01
  * X                      (X) float64 1kB 76.05 76.15 76.25 ... 91.85 91.95
  * Y                      (Y) float64 880B 21.05 21.15 21.25 ... 31.85 31.95
Dimensions without coordinates: bnds
Data variables:
    time_bnds              (time, bnds) datetime64[ns] 4kB ...
    Evap_tavg              (time, Y, X) float32 18MB ...
    Qs_tavg                (time, Y, X) float32 18MB ...
    SWE_inst               (time, Y, X) float32 18MB ...
    Tair_f_tavg            (time, Y, X) float32 18MB ...
    SoilMoi00_10cm_tavg    (time, Y, X) float32 18MB ...
    SoilMoi10_40cm_tavg    (time, Y, X) float32 18MB ...
    SoilMoi40_100cm_tavg   (time, Y, X) float32 18MB ...
    SoilMoi100_200cm_tavg  (time, Y, X) float32 18MB ...
Attributes: (12/20)
    CDI:                       Climate Data Interface version 1.9.8 (https://...
    Conventions:               CF-1.4
    history:                   Fri Oct 30 12:23:53 2020: cdo -z zip_4 replace...
    institution:               NASA GSFC
    missing_value:             -9999.0
    NUM_SOIL_LAYERS:           4
    ...                        ...
    DX:                        0.1
    DY:                        0.1
    nco_openmp_thread_number:  1
    NCO:                       4.4.4
    history_L34RS:             'Created by L34RS v1.4.4 @ NASA GES DISC on Ju...
    CDO:                       Climate Data Operators version 1.9.8 (https://...

## Regridding using dataarray

In [5]:
regrid_lat = np.arange(21.125,32.125,0.25)
regrid_lon = np.arange(76.125,92.125,0.25)
data_array = dataset.interp(Y = regrid_lat, X = regrid_lon,method='slinear')
#data_array - data_array.rename({'X':'lon','Y':'lat'})
data_array

<xarray.Dataset> Size: 45MB
Dimensions:                (time: 251, bnds: 2, Y: 44, X: 64)
Coordinates:
  * time                   (time) datetime64[ns] 2kB 2003-02-01 ... 2023-12-01
  * Y                      (Y) float64 352B 21.12 21.38 21.62 ... 31.62 31.88
  * X                      (X) float64 512B 76.12 76.38 76.62 ... 91.62 91.88
Dimensions without coordinates: bnds
Data variables:
    time_bnds              (time, bnds) datetime64[ns] 4kB 2003-02-01 ... 202...
    Evap_tavg              (time, Y, X) float64 6MB 3.01e-06 ... 8.543e-07
    Qs_tavg                (time, Y, X) float64 6MB 7.997e-10 ... 7.326e-10
    SWE_inst               (time, Y, X) float64 6MB 0.0 0.0 ... 0.2052 0.1048
    Tair_f_tavg            (time, Y, X) float64 6MB 298.8 297.8 ... 262.9 261.0
    SoilMoi00_10cm_tavg    (time, Y, X) float64 6MB 0.1501 0.1748 ... 0.1777
    SoilMoi10_40cm_tavg    (time, Y, X) float64 6MB 0.1491 0.1566 ... 0.2771
    SoilMoi40_100cm_tavg   (time, Y, X) float64 6MB 0.1501 0.157 ... 0.1917
    SoilMoi100_200cm_tavg  (time, Y, X) float64 6MB 0.3851 0.3833 ... 0.2002
Attributes: (12/20)
    CDI:                       Climate Data Interface version 1.9.8 (https://...
    Conventions:               CF-1.4
    history:                   Fri Oct 30 12:23:53 2020: cdo -z zip_4 replace...
    institution:               NASA GSFC
    missing_value:             -9999.0
    NUM_SOIL_LAYERS:           4
    ...                        ...
    DX:                        0.1
    DY:                        0.1
    nco_openmp_thread_number:  1
    NCO:                       4.4.4
    history_L34RS:             'Created by L34RS v1.4.4 @ NASA GES DISC on Ju...
    CDO:                       Climate Data Operators version 1.9.8 (https://...

## Rename x and y to lat and lon if reuired

In [6]:
data_array = data_array.rename({'X':'lon','Y':'lat'})
data_array

<xarray.Dataset> Size: 45MB
Dimensions:                (time: 251, bnds: 2, lat: 44, lon: 64)
Coordinates:
  * time                   (time) datetime64[ns] 2kB 2003-02-01 ... 2023-12-01
  * lat                    (lat) float64 352B 21.12 21.38 21.62 ... 31.62 31.88
  * lon                    (lon) float64 512B 76.12 76.38 76.62 ... 91.62 91.88
Dimensions without coordinates: bnds
Data variables:
    time_bnds              (time, bnds) datetime64[ns] 4kB 2003-02-01 ... 202...
    Evap_tavg              (time, lat, lon) float64 6MB 3.01e-06 ... 8.543e-07
    Qs_tavg                (time, lat, lon) float64 6MB 7.997e-10 ... 7.326e-10
    SWE_inst               (time, lat, lon) float64 6MB 0.0 0.0 ... 0.1048
    Tair_f_tavg            (time, lat, lon) float64 6MB 298.8 297.8 ... 261.0
    SoilMoi00_10cm_tavg    (time, lat, lon) float64 6MB 0.1501 0.1748 ... 0.1777
    SoilMoi10_40cm_tavg    (time, lat, lon) float64 6MB 0.1491 0.1566 ... 0.2771
    SoilMoi40_100cm_tavg   (time, lat, lon) float64 6MB 0.1501 0.157 ... 0.1917
    SoilMoi100_200cm_tavg  (time, lat, lon) float64 6MB 0.3851 0.3833 ... 0.2002
Attributes: (12/20)
    CDI:                       Climate Data Interface version 1.9.8 (https://...
    Conventions:               CF-1.4
    history:                   Fri Oct 30 12:23:53 2020: cdo -z zip_4 replace...
    institution:               NASA GSFC
    missing_value:             -9999.0
    NUM_SOIL_LAYERS:           4
    ...                        ...
    DX:                        0.1
    DY:                        0.1
    nco_openmp_thread_number:  1
    NCO:                       4.4.4
    history_L34RS:             'Created by L34RS v1.4.4 @ NASA GES DISC on Ju...
    CDO:                       Climate Data Operators version 1.9.8 (https://...

# Precipitation

In [ ]:
file_path_nc ="/content/drive/MyDrive/Thesis/RAW/Combined_FLDAS.nc"
dataset = xr.open_dataset(file_path_nc)
dataset